In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [15]:
import csv
import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os
import pandas as pd

# from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

os.environ["CUDA_VISIBLE_DEVICES"] = "3"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()

In [16]:

# path = '/kaggle/input/marathi/mar_train.csv'
# path_val = '/kaggle/input/marathi/mar_valid.csv'
# path_test = '/kaggle/input/marathi/mar_test.csv'

lang = 'hin'
path = f"../../aks_dataset/{lang}/train.csv"
path_val = f"../../aks_dataset/{lang}/valid.csv"
path_test = f"../../aks_dataset/{lang}/test.csv"

In [17]:
df = pd.read_csv(path , header = None)
df_val = pd.read_csv(path_val , header = None)
df_test = pd.read_csv(path_test , header = None)
english_words_val = df_val[0]
marathi_words_val = df_val[1]
english_words_test = df_test[0]
marathi_words_test = df_test[1]
english_words = df[0]
marathi_words = df[1]

In [18]:
# creating list of charecters in both languages

english_char_list = []
max_length_word_english = -1
for word in english_words:
  max_length_word_english = max(max_length_word_english,len(word)) 
  for char in word :
    english_char_list.append(char);
english_char_list = list(set(english_char_list))
english_char_list.sort()

marathi_char_list = []
max_length_word_marathi = -1
for word in marathi_words:
  max_length_word_marathi = max(max_length_word_marathi,len(word))
  for char in word :
    marathi_char_list.append(char);
marathi_char_list = list(set(marathi_char_list))
marathi_char_list.sort()


# finding out the maximum size word for english and marathi from validation and test data.
for word in english_words_val:
  max_length_word_english = max(max_length_word_english,len(word))
for word in english_words_test:
  max_length_word_english = max(max_length_word_english,len(word)) 
for word in marathi_words_val:
  max_length_word_marathi = max(max_length_word_marathi,len(word))
for word in marathi_words_test:
  max_length_word_marathi = max(max_length_word_marathi,len(word))


In [7]:
max_length_word_english
max_length_word_marathi

26

In [8]:
max_length_word_marathi

26

In [9]:
import pickle
# Save to pickle file
with open('models/marathi_char_list.pkl', 'wb') as f:
    pickle.dump(marathi_char_list, f)

In [10]:
import pickle
# Save to pickle file
with open('models/english_char_list.pkl', 'wb') as f:
    pickle.dump(english_char_list, f)

In [10]:
# english word to vector size = 27 ie. max_length_word_english
# marathi word to vector size = 20 ie. max_length_word_marathi
# for one word.
def word2vec(word, lang):
  vec = []
  if(lang == "english"):
    vec.append(len(english_char_list) + 1)
    for char in word:
      for i in range(len(english_char_list)):
        if(english_char_list[i] == char):
          vec.append(i+1)
    while(len(vec) < max_length_word_english + 1): # padding with max_length + 1.
        vec.append(0)
    vec.append(0)
  else :
    vec.append(len(marathi_char_list) + 1)
    for char in word:
      for i in range(len(marathi_char_list)):
        if( marathi_char_list[i] == char):
          vec.append(i+1)
    while(len(vec) < max_length_word_marathi + 1):  # padding with max_length + 1.
        vec.append(0)
    vec.append(0)
  return(vec)


In [12]:
vec = word2vec('hi',"marathi")
print('hi')
print(vec)

hi
[65, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [13]:
# creating matrix of representation for whole words of english and marathi.

def ip_matrix_construct(words, lang):
  ans = []
  for word in words:
    ans.append(word2vec(word, lang))
  return(ans)

In [14]:
# calculated representations of whole english and marathi words in variables english and marathi matrix.
english_matrix = ip_matrix_construct(english_words, "english")
marathi_matrix = ip_matrix_construct(marathi_words, "marathi")
english_matrix = torch.tensor(english_matrix)
marathi_matrix = torch.tensor(marathi_matrix)

english_matrix_val = ip_matrix_construct(english_words_val, "english")
marathi_matrix_val = ip_matrix_construct(marathi_words_val, "marathi")
english_matrix_val = torch.tensor(english_matrix_val)
marathi_matrix_val = torch.tensor(marathi_matrix_val)
english_matrix_test = ip_matrix_construct(english_words_test, "english")
marathi_matrix_test =ip_matrix_construct(marathi_words_test, "marathi")
english_matrix_test = torch.tensor(english_matrix_test)
marathi_matrix_test = torch.tensor(marathi_matrix_test)

In [2]:
class Encoder(nn.Module):
  def __init__(self,input_size, embedding_size, hidden_size, enc_layers, p, cell_type, bidirectional):
    super(Encoder,self).__init__()
    self.hidden_size = hidden_size
    self.enc_layers = enc_layers
    self.dropout = nn.Dropout(p)
    self.cell_type = cell_type
    self.bidirectional = bidirectional
    self.embedding = nn.Embedding(input_size, embedding_size)
    if(cell_type == "GRU"):
      self.gru = nn.GRU(embedding_size, hidden_size, enc_layers, dropout = p, bidirectional = bidirectional)
    if(cell_type == "RNN"):
      self.rnn = nn.RNN(embedding_size, hidden_size, enc_layers, dropout = p, bidirectional = bidirectional)
    if(cell_type == "LSTM"):
      self.lstm = nn.LSTM(embedding_size, hidden_size, enc_layers, dropout = p, bidirectional = bidirectional)

  def forward(self, x):
    embedding = self.dropout(self.embedding(x))
    if(self.cell_type == "GRU"):
      output, hidden = self.gru(embedding)
    if(self.cell_type == "RNN"):
      output, hidden = self.rnn(embedding)
    if(self.cell_type == "LSTM"):
      outputs, (hidden,cell) = self.lstm(embedding)
      return outputs, hidden, cell
    return output, hidden

  def initHidden(self):
    return torch.zeros(1, 1, self.hidden_size, device=device)

In [3]:
class Decoder(nn.Module):
  def __init__(self, input_size, embedding_size, hidden_size, output_size, dec_layers, p, cell_type):
    super(Decoder, self).__init__()
    self.hidden_size = hidden_size
    self.dec_layers = dec_layers
    self.dropout = nn.Dropout(p)
    self.cell_type = cell_type
    self.embedding = nn.Embedding(input_size, embedding_size)
    if(cell_type == "GRU"):
      self.gru = nn.GRU(embedding_size, hidden_size, dec_layers, dropout = p)
    if(cell_type == "RNN"):
      self.rnn = nn.RNN(embedding_size, hidden_size, dec_layers, dropout = p)
    if(cell_type == "LSTM"):
      self.lstm = nn.LSTM(embedding_size, hidden_size, dec_layers, dropout = p)
    self.fc = nn.Linear(hidden_size, output_size)  # fully connected.
  
  def forward(self,x,output, hidden, cell = 0):
    x = x.unsqueeze(0).int()
    embedding = self.dropout(self.embedding(x))
    if(self.cell_type == "GRU"):
        outputs, hidden = self.gru(embedding, hidden)
    if(self.cell_type == "RNN"):
        outputs, hidden = self.rnn(embedding, hidden)
    if(self.cell_type == "LSTM"):
        outputs, (hidden, cell) = self.lstm(embedding, (hidden, cell))
    # shape of outputs: (1, N, hidden_size)
    predictions = self.fc(outputs)
    # shape of predictions: (1, N, length_of_vocab)
    predictions = predictions.squeeze(0)
    # shape of predictions: (N, length_of_vocab)
    if(self.cell_type == "LSTM"):
        return predictions, hidden, cell
    return predictions, hidden


  def initHidden(self):
    return torch.zeros(1, 1, self.hidden_size, device=device)

In [4]:
class Atten_decoder(nn.Module):
  def __init__(self, input_size, embedding_size, hidden_size, output_size, dec_layers, p, cell_type, bidirectional):
    super(Atten_decoder, self).__init__()
    self.hidden_size = hidden_size
    self.output_size = output_size
    self.max_length = len(english_matrix[0])  
    self.dec_layers = dec_layers
    self.dropout = nn.Dropout(p)
    self.cell_type = cell_type
    self.embedding = nn.Embedding(input_size, embedding_size)
    if(cell_type == "GRU"):
      self.gru = nn.GRU(hidden_size, hidden_size, dec_layers, dropout = p)
    if(cell_type == "RNN"):
      self.rnn = nn.RNN(hidden_size, hidden_size, dec_layers, dropout = p)
    if(cell_type == "LSTM"):
      self.lstm = nn.LSTM(hidden_size, hidden_size, dec_layers, dropout = p)
    self.fc = nn.Linear(hidden_size, output_size)  # fully connected.
    self.attn = nn.Linear(hidden_size+embedding_size, self.max_length)
    if(bidirectional):
      self.attn_combine = nn.Linear(hidden_size * 2 + embedding_size, hidden_size)
    else :
      self.attn_combine = nn.Linear(hidden_size + embedding_size, hidden_size)

  def forward(self, x,output, hidden, cell = 0):
    x = x.unsqueeze(0)
    output=output.permute(1,0,2)
    embedded = self.embedding(x)
    embedded = self.dropout(embedded)
    attn_weights = F.softmax(self.attn(torch.cat((embedded[0],hidden[0]), 1)), dim = 1)
    attn_applied = torch.bmm(attn_weights.unsqueeze(1),output)
    attn_applied = attn_applied.squeeze(1)
    op = torch.cat((embedded[0], attn_applied), 1)

    op = self.attn_combine(op).unsqueeze(0)
    op = F.relu(op)
    if(self.cell_type == "GRU"):
        outputs, hidden = self.gru(op, hidden)
    if(self.cell_type == "RNN"):
        outputs, hidden = self.rnn(op, hidden)
    if(self.cell_type == "LSTM"):
        outputs, (hidden, cell) = self.lstm(op, (hidden, cell))
    predictions = self.fc(outputs)
    predictions = predictions.squeeze(0)
    if(self.cell_type == "LSTM"):
        return predictions, hidden, cell
    return predictions, hidden



In [5]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, cell_type, bidirectional, enc_layers, dec_layers):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.cell_type = cell_type
        self.bidirectional = bidirectional
        self.enc_layers = enc_layers
        self.dec_layers = dec_layers

    def forward(self, source, target, teacher_force_ratio=0.5):
        batch_size = source.shape[1]
        target_len = target.shape[0]
        target_vocab_size = len(marathi_char_list) + 2  
        outputs = torch.zeros(target_len, batch_size, target_vocab_size).to(device)
        if(self.cell_type == "LSTM"):
            encoder_output, hidden, cell = self.encoder(source)
        else:
            encoder_output, hidden = self.encoder(source)
        if(self.enc_layers != self.dec_layers or self.bidirectional == True):
          hidden = hidden[self.enc_layers - 1] + hidden[self.enc_layers - 1]
          hidden = hidden.repeat(self.dec_layers,1,1)
          if(self.cell_type == "LSTM"):
              cell = cell[self.enc_layers - 1] + cell[self.enc_layers - 1]
              cell = cell.repeat(self.dec_layers,1,1)
        
        x = target[0]
    
        for t in range(1, target_len):
            if(self.cell_type == "LSTM"):
                output, hidden, cell = self.decoder(x, encoder_output, hidden, cell)
            else :
                output, hidden = self.decoder(x, encoder_output, hidden)
            outputs[t] = output

            best_guess = output.argmax(1)

            x = target[t] if random.random() < teacher_force_ratio else best_guess
            
        return outputs


In [6]:
def Accuracy(model, english_matrix, marathi_matrix, epoch, batch_size):
    correct_count = 0
    for batch_idx in range((int)(len(english_matrix) / batch_size)):
        inp_data = english_matrix[batch_size * batch_idx : batch_size * (batch_idx+1)].to(device)
        target = marathi_matrix[batch_size * batch_idx : batch_size * (batch_idx+1)].to(device)
        output = model.forward(inp_data.T, target.T, 0)
        output = nn.Softmax(dim=2)(output)
        output = torch.argmax(output, dim=2)
        output = output.T
        for i in range(batch_size):
            if(torch.equal(output[i][1:],target[i][1:])):
                correct_count += 1
    accuracy = correct_count * 100 / len(english_matrix)
    return accuracy
        

In [27]:
# creating list of expected marathi word and predicted marathi word.
predictions_vanilla_train = []
predictions_vanilla_val = []
predictions_vanilla_test = []
def matrix_to_words(model, english_matrix, marathi_matrix, batch_size, data_type):
  for batch_idx in range((int)(len(english_matrix) / batch_size)+1):
        inp_data = english_matrix[batch_size * batch_idx : batch_size * (batch_idx+1)].to(device)
        target = marathi_matrix[batch_size * batch_idx : batch_size * (batch_idx+1)].to(device)
        output = model.forward(inp_data.T, target.T, 0)
        output = nn.Softmax(dim=2)(output)
        output = torch.argmax(output, dim=2)
        output = output.T
        for i in range(len(target)):
          target_word = target[i]
          output_word = output[i]
          word1 = ""
          word2 = ""
          for j in range(len(target_word)):
            if(target_word[j]>0 and target_word[j]<64):
              word1 += marathi_char_list[target_word[j] - 1]
          for j in range(len(output_word)):
            if(output_word[j]>0 and output_word[j]<64):
              word2 += marathi_char_list[output_word[j] - 1]
          temp = [word1, word2]
          if(data_type == "train"):
            predictions_vanilla_train.append(temp)
          if(data_type == "validation"):
            predictions_vanilla_val.append(temp)
          if(data_type == "test"):
            predictions_vanilla_test.append(temp)

In [16]:
def neural_network(cell_type, bidirectional, enc_layers, dec_layers, batch_size, embedding_size, hidden_size, enc_dropout, dec_dropout, attention):
  learning_rate = 1e-3                                                                                                                                                                                                                                                              
  num_epochs = 20
  input_size_encoder = len(english_char_list) + 2  
  input_size_decoder = len(marathi_char_list) + 2  
  output_size        = len(marathi_char_list) + 2  

  encoder_net = Encoder(input_size_encoder, embedding_size, hidden_size, enc_layers, enc_dropout, cell_type,bidirectional).to(device)
  if(attention):
    decoder_net = Atten_decoder(input_size_decoder,embedding_size,hidden_size,output_size,dec_layers,dec_dropout, cell_type, bidirectional).to(device)
  else:
    decoder_net = Decoder(input_size_decoder,embedding_size,hidden_size,output_size,dec_layers,dec_dropout, cell_type).to(device)

  model = Seq2Seq(encoder_net, decoder_net, cell_type, bidirectional, enc_layers, dec_layers).to(device)
  optimizer = optim.Adam(model.parameters(), lr=learning_rate)

  pad_idx = len(marathi_char_list) + 1  # 64 # pading index for marathi
  criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)


  for epoch in range(num_epochs):
      print("epoch = ",epoch)

      model.train()
      total_loss = 0
      step = 0
      for batch_idx in range((int)(len(english_matrix) / batch_size)):

          inp_data = english_matrix[batch_size * batch_idx : batch_size * (batch_idx+1)].to(device)
          target = marathi_matrix[batch_size * batch_idx : batch_size * (batch_idx+1)].to(device)
          target = target.T
          output = model(inp_data.T, target)

          output = output[1:].reshape(-1, output.shape[2])
          target = target[1:].reshape(-1)
          optimizer.zero_grad()
          loss = criterion(output, target)
          total_loss += loss
          loss.backward()

          torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

          optimizer.step()

          step += 1
      print("total loss = ",total_loss / step)
      training_accuracy = Accuracy(model, english_matrix, marathi_matrix, epoch, batch_size)
      print("Training Accuracy = ", training_accuracy)
      val_accuracy = Accuracy(model, english_matrix_val, marathi_matrix_val, epoch, batch_size)
      print("Validation accuracy = ",val_accuracy)
      if(epoch > 10 and training_accuracy < 10):
        return
      if(epoch > 5 and training_accuracy < 1):
        return # useless code
  matrix_to_words(model, english_matrix, marathi_matrix, batch_size, "train")
  print("train done")
  matrix_to_words(model, english_matrix_val, marathi_matrix_val, batch_size, "validation")
  print("val done")
  matrix_to_words(model, english_matrix_test, marathi_matrix_test, batch_size, "test")
  return model

In [29]:
cell_type = "LSTM"
bidirectional = True
enc_layers = 2
dec_layers = 2
batch_size = 256
embedding_size = 256
hidden_size = 512
enc_dropout = 0
dec_dropout = 0
attention = True


model = neural_network(cell_type, bidirectional, enc_layers, dec_layers, batch_size, embedding_size, hidden_size, enc_dropout, dec_dropout, attention)

In [18]:
# creating csv file of predictions
filename_train  = 'bi_predictions_vanilla_train_attention.csv'
filename_val = 'bi_predictions_vanilla_val_attention.csv'
filename_test = 'bi_predictions_vanilla_test_attention.csv'

with open(filename_train, 'w', newline='') as file:
    writer = csv.writer(file, quoting=csv.QUOTE_NONE)
    writer.writerows(predictions_vanilla_train)

print(f'CSV file "{filename_train}" has been created.')


with open(filename_val, 'w', newline='') as file:
    writer = csv.writer(file, quoting=csv.QUOTE_NONE)
    writer.writerows(predictions_vanilla_val)

print(f'CSV file "{filename_val}" has been created.')


with open(filename_test, 'w', newline='') as file:
    writer = csv.writer(file, quoting=csv.QUOTE_NONE)
    writer.writerows(predictions_vanilla_test)

print(f'CSV file "{filename_test}" has been created.')

CSV file "bi_predictions_vanilla_train_attention.csv" has been created.
CSV file "bi_predictions_vanilla_val_attention.csv" has been created.
CSV file "bi_predictions_vanilla_test_attention.csv" has been created.


In [30]:
matrix_to_words(model, english_matrix_test, marathi_matrix_test, batch_size, "test")

In [31]:
import csv

# --- define data ---
filename_test = 'predictions/bi_attn.csv'

# english_words: list of English strings
# predictions_vanilla_test: list of [actual_word, predicted_word] pairs



# Combine into rows: [English, Actual, Predicted]
rows = []
for eng, (actual, predicted) in zip(english_words_test, predictions_vanilla_test):
    rows.append([eng, actual, predicted])

# --- write to CSV ---
with open(filename_test, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['eng', 'actual', 'predicted'])  # header
    writer.writerows(rows)

print(f'✅ CSV file \"{filename_test}\" has been created successfully.')

✅ CSV file "predictions/bi_attn.csv" has been created successfully.


In [56]:
import pandas as pd

def calculate_accuracies(csv_path):
    # Read CSV (no headers)
    df = pd.read_csv(csv_path, header=None, sep=',', names=['actual', 'predicted'])
    
    # Drop any empty rows
    df = df.dropna()
    
    total_words = len(df)
    correct_words = 0
    total_chars = 0
    correct_chars = 0

    for _, row in df.iterrows():
        actual = str(row['actual']).strip()
        predicted = str(row['predicted']).strip()

        # --- Word-level accuracy ---
        if actual == predicted:
            correct_words += 1

        # --- Character-level accuracy ---
        max_len = max(len(actual), len(predicted))
        total_chars += max_len
        for a, b in zip(actual, predicted):
            if a == b:
                correct_chars += 1

    word_acc = correct_words / total_words * 100 if total_words > 0 else 0
    char_acc = correct_chars / total_chars * 100 if total_chars > 0 else 0

    print(f"✅ Word-level accuracy: {word_acc:.2f}%")
    print(f"✅ Character-level accuracy: {char_acc:.2f}%")

    return word_acc, char_acc

In [57]:
csv_path = "bi_predictions_vanilla_test_attention.csv"
calculate_accuracies(csv_path)

✅ Word-level accuracy: 43.16%
✅ Character-level accuracy: 73.80%


(43.15905448717949, 73.79789660609586)

In [58]:
# Save the entire model
torch.save(model, f"model_{cell_type}_{'bi' if bidirectional else 'uni'}_{'attn' if attention else 'noattn'}.pt")
print("Model saved successfully!")

Model saved successfully!


In [7]:
model = torch.load("models/model_LSTM_bi_attn.pt")
model.eval()  # Important: set to eval mode before testing

Seq2Seq(
  (encoder): Encoder(
    (dropout): Dropout(p=0, inplace=False)
    (embedding): Embedding(28, 256)
    (lstm): LSTM(256, 512, num_layers=2, bidirectional=True)
  )
  (decoder): Atten_decoder(
    (dropout): Dropout(p=0, inplace=False)
    (embedding): Embedding(66, 256)
    (lstm): LSTM(512, 512, num_layers=2)
    (fc): Linear(in_features=512, out_features=66, bias=True)
    (attn): Linear(in_features=768, out_features=31, bias=True)
    (attn_combine): Linear(in_features=1280, out_features=512, bias=True)
  )
)

In [60]:
import pandas as pd

def calculate_accuracies(csv_path):
    # Read CSV (no headers)
    df = pd.read_csv(csv_path, header=None, sep=',', names=['actual', 'predicted'])
    
    # Drop any empty rows
    df = df.dropna()
    
    total_words = len(df)
    correct_words = 0
    total_chars = 0
    correct_chars = 0

    for _, row in df.iterrows():
        actual = str(row['actual']).strip()
        predicted = str(row['predicted']).strip()

        # --- Word-level accuracy ---
        if actual == predicted:
            correct_words += 1

        # --- Character-level accuracy ---
        max_len = max(len(actual), len(predicted))
        total_chars += max_len
        for a, b in zip(actual, predicted):
            if a == b:
                correct_chars += 1

    word_acc = correct_words / total_words * 100 if total_words > 0 else 0
    char_acc = correct_chars / total_chars * 100 if total_chars > 0 else 0

    print(f"✅ Word-level accuracy: {word_acc:.2f}%")
    print(f"✅ Character-level accuracy: {char_acc:.2f}%")

    return word_acc, char_acc

In [19]:
import torch
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# === 1. Load saved model ===
model_path = "models/model_LSTM_bi_attn.pt"   # <-- change filename as per your saved one
model = torch.load(model_path, map_location=device)
model.eval()
print("✅ Model loaded successfully")

# === 2. Helper function to convert English word to tensor ===
def encode_word(word, lang_dict, max_len):
    tensor = torch.zeros(max_len, dtype=torch.long)
    for i, ch in enumerate(word):
        if ch in lang_dict:
            tensor[i] = lang_dict[ch] + 1
        else:
            tensor[i] = 0  # unknown char or padding
    return tensor.unsqueeze(1).to(device)  # shape: (seq_len, 1)

# === 3. Helper function to decode tensor back to Marathi string ===
def decode_tensor(tensor, marathi_char_list):
    word = ""
    for idx in tensor:
        if 0 < idx < len(marathi_char_list) + 1:
            word += marathi_char_list[idx - 1]
    return word

# === 2. Predict for a single English word ===
def predict_word(model, english_word):
    # convert to numeric tensor using your own function
    eng_vec = word2vec(english_word, "english")
    inp_tensor = torch.tensor(eng_vec, dtype=torch.long).unsqueeze(1).to(device)  # shape [seq_len, 1]
    
    # dummy target tensor (zeroed out)
    target_tensor = torch.zeros(max_length_word_marathi + 2, 1, dtype=torch.long).to(device)
    
    with torch.no_grad():
        output = model.forward(inp_tensor, target_tensor, 0)  # teacher forcing = 0
        output = nn.Softmax(dim=2)(output)
        output = torch.argmax(output, dim=2).squeeze(1)  # shape: [seq_len]
    
    # decode the predicted indices into Marathi characters
    predicted_word = ""
    for idx in output:
        idx = idx.item()
        if 0 < idx <= len(marathi_char_list):
            predicted_word += marathi_char_list[idx - 1]
    
    return predicted_word

✅ Model loaded successfully


In [20]:
# === 5. Example usage ===
english_word = "kaustubh"  # any test English transliteration
predicted_marathi = predict_word(
    model, 
    english_word
)
print(f"English: {english_word}")
print(f"Predicted Marathi: {predicted_marathi}")

English: kaustubh
Predicted Marathi: कौस्तुभ


In [23]:
def predict_with_sos(model, english_word, word2vec, max_length_word_marathi, marathi_char_list, SOS_IDX=65, device='cpu'):
    model.eval()
    with torch.no_grad():
        eng_vec = word2vec(english_word, "english")
        inp_tensor = torch.tensor(eng_vec, dtype=torch.long).unsqueeze(1).to(device)   # (seq_len, 1)

        target_len = max_length_word_marathi
        target_tensor = torch.zeros(target_len, 1, dtype=torch.long).to(device)
        target_tensor[0, 0] = SOS_IDX   # <-- important

        if hasattr(model, "reset_hidden"):
            model.reset_hidden(batch_size=1)

        logits = model.forward(inp_tensor, target_tensor, 0)    # (seq_len_dec, 1, vocab)
        preds = torch.argmax(torch.softmax(logits, dim=2), dim=2).squeeze(1).cpu().tolist()

        # robust decoding: skip leading special tokens (PAD or SOS) then decode until EOS if present
        PAD_IDX = 0
        EOS_IDX = None  # set if you know it
        # skip leading PAD/SOS
        i = 0
        while i < len(preds) and (preds[i] == PAD_IDX or preds[i] == SOS_IDX):
            i += 1
        decoded = []
        for idx in preds[i:]:
            if EOS_IDX is not None and idx == EOS_IDX:
                break
            if 0 < idx <= len(marathi_char_list):
                decoded.append(marathi_char_list[idx - 1])
        predicted_word = "".join(decoded)
    return predicted_word

In [24]:
import pandas as pd
import torch
from tqdm import tqdm

# --- Load the test CSV (no headers) ---
original_csv = "../../aks_dataset/hin/test.csv"
df_test = pd.read_csv(original_csv, header=None)

# --- Give column names to df_test ---
df_test.columns = ['eng', 'actual']  # first column = English, second column = reference Hindi

# --- Setup device ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Make sure your model is on the device
model.to(device)
model.eval()  # Set to evaluation mode

# --- Predict function wrapper ---
def predict_words_batch(model, words):
    """Predict multiple words using GPU."""
    predictions = []
    with torch.no_grad():
        for word in tqdm(words):
            word = str(word).strip()
            pred = predict_with_sos( 
    model, word, word2vec, max_length_word_marathi, marathi_char_list, 65, device
)  # your existing function
            predictions.append(pred)
    return predictions

# --- Get first column (English words) ---
first_column = df_test['eng'].tolist()

# --- Run predictions ---
predicted = predict_words_batch(model, first_column)

# --- Save predictions back to DataFrame ---
df_test['predicted'] = predicted

# --- Save to CSV ---
df_test.to_csv("predictions/greedy/bi_attn.csv", index=False)
print("Predictions saved to bi_lstm_predictions.csv")

Using device: cuda


100%|█████████████████████████████████████████████████████████████████████████████| 10112/10112 [02:54<00:00, 58.00it/s]


Predictions saved to bi_lstm_predictions.csv


In [ ]:
import pandas as pd
import torch
from tqdm import tqdm

# --- Load the test CSV (no headers) ---
original_csv = "../../aks_dataset/hin/test.csv"
df_test = pd.read_csv(original_csv, header=None)

# --- Give column names to df_test ---
df_test.columns = ['eng', 'actual']  # first column = English, second column = reference Hindi

# --- Setup device ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Make sure your model is on the device
model.to(device)
model.eval()  # Set to evaluation mode

# --- Predict function wrapper ---
def predict_words_batch(model, words):
    """Predict multiple words using GPU."""
    predictions = []
    with torch.no_grad():
        for word in tqdm(words):
            word = str(word).strip()
            pred, beams = beam_search_predict(
    model=model,
    english_word=word,
    word2vec=word2vec,
    max_length_word_marathi=max_length_word_marathi,
    marathi_char_list=marathi_char_list,
    SOS_IDX=SOS_IDX,            # set this from your training config / Counter output
    device=device,
    beam_width=4,
    EOS_IDX=None,            # set to your EOS index if known, otherwise None
    length_norm_alpha=0.0,
    return_beams=True
)  # your existing function
            predictions.append(pred)
    return predictions

# --- Get first column (English words) ---
first_column = df_test['eng'].tolist()

# --- Run predictions ---
predicted = predict_words_batch(model, first_column)

# --- Save predictions back to DataFrame ---
df_test['predicted'] = predicted

# --- Save to CSV ---
df_test.to_csv("predictions/beam/uni_attn.csv", index=False)
print("Predictions saved to bi_lstm_predictions.csv")

In [25]:
import torch
import torch.nn.functional as F
from heapq import nlargest
from typing import List, Tuple, Optional

def beam_search_predict(
    model,
    english_word: str,
    word2vec,
    max_length_word_marathi: int,
    marathi_char_list: List[str],
    SOS_IDX: int,
    device: torch.device,
    beam_width: int = 4,
    PAD_IDX: int = 0,
    EOS_IDX: Optional[int] = None,
    length_norm_alpha: float = 0.0,   # 0.0 -> no length norm; >0 gives light normalization
    return_beams: bool = False,
):
    """
    Beam-search inference for a single example using model.forward(inp, target, 0)
    - model.forward expects (seq_len_enc, batch, ...) for inp and (seq_len_dec, batch) for target
    - Works for batch size = 1
    - beam_width: number of beams kept
    - length_norm_alpha: if >0, use (score / (length**alpha)) scoring for tie-breaking
    - EOS_IDX: optional; if provided the beam ends when EOS is generated
    Returns: (predicted_word, beams) if return_beams else predicted_word
    Beams format: list of tuples (token_list, score)
    """
    model.eval()
    with torch.no_grad():
        # prepare encoder input
        eng_vec = word2vec(english_word, "english")
        inp_tensor = torch.tensor(eng_vec, dtype=torch.long).unsqueeze(1).to(device)  # (seq_len_enc, 1)

        # reset model state if applicable
        if hasattr(model, "reset_hidden"):
            try:
                model.reset_hidden(batch_size=1)
            except TypeError:
                model.reset_hidden()

        # beam represented as tuple (token_list, logprob_sum, finished_flag)
        # token_list includes SOS at position 0
        init_beam = ([SOS_IDX], 0.0, False)
        beams: List[Tuple[List[int], float, bool]] = [init_beam]

        for step in range(1, max_length_word_marathi):
            all_candidates: List[Tuple[List[int], float, bool]] = []

            # expand each beam
            for tokens, logprob, finished in beams:
                if finished:
                    # carry over finished beams unchanged
                    all_candidates.append((tokens, logprob, True))
                    continue

                # build target tensor for this partial sequence
                tgt_tensor = torch.tensor(tokens, dtype=torch.long).unsqueeze(1).to(device)  # (cur_len, 1)
                # forward - we only need logits at last timestep
                logits = model.forward(inp_tensor, tgt_tensor, 0)   # (seq_len_dec, batch=1, vocab)
                last_logits = logits[-1, 0, :]                      # (vocab,)
                log_probs = F.log_softmax(last_logits, dim=0)       # (vocab,)

                # pick top-k token candidates for expansion (local pruning)
                topk_vals, topk_idx = torch.topk(log_probs, k=min(beam_width, log_probs.size(0)))
                topk_vals = topk_vals.cpu().tolist()
                topk_idx = topk_idx.cpu().tolist()

                for v, idx_token in zip(topk_vals, topk_idx):
                    new_tokens = tokens + [int(idx_token)]
                    new_logprob = logprob + float(v)
                    is_finished = (EOS_IDX is not None and idx_token == EOS_IDX)
                    all_candidates.append((new_tokens, new_logprob, is_finished))

            # select top beam_width candidates by score (apply optional length norm for ranking)
            def score_for_ranking(candidate):
                token_list, s, finished_flag = candidate
                # length used for normalization: exclude SOS
                length = max(1, len(token_list) - 1)
                if length_norm_alpha > 0.0:
                    return s / (length ** length_norm_alpha)
                return s

            # keep the top-k candidates
            beams = nlargest(beam_width, all_candidates, key=score_for_ranking)

            # if all beams finished early, stop
            if all(f for (_, _, f) in beams):
                break

        # choose final best beam: prefer finished beams (if any)
        finished_beams = [b for b in beams if b[2]]
        if finished_beams:
            best_beam = max(finished_beams, key=lambda b: score_for_ranking(b))
        else:
            # fallback: choose best by raw score (or normalized)
            best_beam = max(beams, key=lambda b: score_for_ranking(b))

        best_tokens, best_logprob, _ = best_beam

        # robust decoding: drop leading PAD/SOS then decode until EOS (if provided)
        i = 0
        while i < len(best_tokens) and (best_tokens[i] == PAD_IDX or best_tokens[i] == SOS_IDX):
            i += 1

        decoded_chars = []
        for idx in best_tokens[i:]:
            if EOS_IDX is not None and idx == EOS_IDX:
                break
            if 0 < idx <= len(marathi_char_list):
                decoded_chars.append(marathi_char_list[idx - 1])

        predicted_word = "".join(decoded_chars)

        if return_beams:
            # produce readable beams: (token_list, normalized_score)
            readable = []
            for tokens, s, finished in beams:
                norm_score = s / (max(1, len(tokens) - 1) ** length_norm_alpha) if length_norm_alpha > 0 else s
                readable.append((tokens, float(norm_score), finished))
            return predicted_word, readable

        return predicted_word

In [27]:
import pandas as pd
import torch
from tqdm import tqdm

# --- Load the test CSV (no headers) ---
original_csv = "../../aks_dataset/hin/test.csv"
df_test = pd.read_csv(original_csv, header=None)

# --- Give column names to df_test ---
df_test.columns = ['eng', 'actual']  # first column = English, second column = reference Hindi

# --- Setup device ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Make sure your model is on the device
model.to(device)
model.eval()  # Set to evaluation mode

# --- Predict function wrapper ---
def predict_words_batch(model, words):
    """Predict multiple words using GPU."""
    predictions = []
    with torch.no_grad():
        for word in tqdm(words):
            word = str(word).strip()
            pred, beams = beam_search_predict(
    model=model,
    english_word=word,
    word2vec=word2vec,
    max_length_word_marathi=max_length_word_marathi,
    marathi_char_list=marathi_char_list,
    SOS_IDX=65,            # set this from your training config / Counter output
    device=device,
    beam_width=2,
    EOS_IDX=None,            # set to your EOS index if known, otherwise None
    length_norm_alpha=0.0,
    return_beams=True
)  # your existing function
            predictions.append(pred)
    return predictions

# --- Get first column (English words) ---
first_column = df_test['eng'].tolist()

# --- Run predictions ---
predicted = predict_words_batch(model, first_column)

# --- Save predictions back to DataFrame ---
df_test['predicted'] = predicted

# --- Save to CSV ---
df_test.to_csv("predictions/beam/bi_attn.csv", index=False)
print("Predictions saved to bi_lstm_predictions.csv")






Using device: cuda


  5%|███▉                                                                         | 524/10112 [04:17<1:18:28,  2.04it/s]


KeyboardInterrupt: 